In [ ]:
!pip install scrapy

In [ ]:
!apt install libicu-dev pkg-config

In [ ]:
!pip install ICU-Tokenizer

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ICU-Tokenizer: filename=icu_tokenizer-0.0.1-py3-none-any.whl size=13263 sha256=5072fb3ea5e27bbcdc1ee6fe3427963572604aa339031944ac359f0a29ec4203
  Stored in directory: /root/.cache/pip/wheels/1b/ab/eb/d76a3e1fde6a7fb77e0ca3d8043f27d570b77f2b13420b478d
  Created wheel for PyICU: filename=PyICU-2.11-cp310-cp310-linux_x86_64.whl size=1742568 sha256=ab3347bfe1c1bd84022631578b6a0a9570d459acb3e4ce2dfcb994aa5030c3e6
  Stored in directory: /root/.cache/pip/wheels/67/c3/00/2176cc05d3ea22935a9c78f1034b1a3e3697ef11ffb5cbe2f5
Successfully built ICU-Tokenizer PyICU


In [ ]:
from pathlib import Path
import os
import json
from google.colab import drive
import requests
from scrapy import Spider, Request
from scrapy.crawler import CrawlerProcess
from icu_tokenizer import SentSplitter
import yaml
from tqdm import tqdm

In [ ]:
drive.mount("/content/drive", force_remount=True)
os.chdir("/content/drive/MyDrive/corpora/speech-corpora/speech-text/")

Mounted at /content/drive


In [ ]:
SPLITTER = SentSplitter("ff")

In [ ]:
class MySpider(Spider):
    name = "bible"
    with open("extra/links.yml") as links_file:
        links = yaml.safe_load(links_file)
    languages, start_urls = zip(*links.items())
    codes = [url.split(".")[-1] for url in start_urls]
    mapping = dict(zip(codes, languages))

    def download_audio(self, audio_src, output_file):
        data = requests.get(audio_src)
        with open(output_file, 'wb') as out_mp3:
            out_mp3.write(data.content)
            return True, Path(output_file).stem

    def parse(self, response):
        content_to_pass = {"ChapterContent_r___3KRx", "ChapterContent_label__R2PLt", "ChapterContent_note__YlDW0",
                           "ChapterContent_fr__0KsID", "ChapterContent_body__O3qjr", "ft", "w"}
        title = response.css("h1::text")
        title = title.get()
        book, chapter, code = response.url.split("/")[-1].split(".")[-3:]
        language = MySpider.mapping[code]

        output_folder = Path(f"bible/raw/{language}")
        output_folder.mkdir(exist_ok=True, parents=True)
        output_filename = output_folder / f"{book}_{chapter}_{code}"

        with open(f"bible/{code}.books", "a+") as titles:
            titles.write(f"{book}\t{chapter}\t{title}\n")
        audio = response.css("div.pli-1:nth-child(4) > div:nth-child(1) > audio:nth-child(1)")
        if "src" in audio.attrib:
            audio = audio.attrib["src"]
            self.download_audio(audio, f"{output_filename}.mp3")
        with open(f"{output_filename}.txt", "w") as output_file:
            output_file.write(f"{title}\n")
            for content in response.css("div.ChapterContent_chapter__uvbXo div"):
                if content.attrib["class"] in content_to_pass:
                    continue
                spans = content.css("span span")
                if not spans:
                    text = " ".join(verse.strip() for verse in content.css("span *::text").getall())
                    text = text.strip()
                    if not text:
                        continue
                    for sent in SPLITTER.split(text):
                        output_file.write(f"{sent}\n")
                    continue
                verses = []
                for span in spans:
                    if span.attrib["class"] in content_to_pass:
                        continue
                    for verse in span.css("*::text").getall():
                        verses.append(verse.strip())
                text = " ".join(verses).strip()
                text = text.strip()
                if not text:
                    continue
                for sent in SPLITTER.split(text):
                    output_file.write(f"{sent}\n")
        next_page = response.css("div.\[pointer-events\:all\]:nth-child(2) > a:nth-child(1)")
        if next_page:
            yield Request(url=f"https://www.bible.com{next_page.attrib['href']}")

In [ ]:
%%capture
process = CrawlerProcess()

process.crawl(MySpider, name="bible")
process.start()

In [12]:
%cd bible/raw/

/content/drive/MyDrive/corpora/speech-corpora/speech-text/bible/raw


In [17]:
!ls

adamawa  borgu	bororro  caka  liptako	maacina  pular


In [28]:
%cd pular/

/content/drive/MyDrive/corpora/speech-corpora/speech-text/bible/raw/pular


In [29]:
!for f in $( find . -type f -name "*.mp3" ); do ffmpeg -i "$f" -ac 1 -ar 16000 "${f%mp3}wav" ; done
%cd ..

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
    Metadata:
      encoder         : Lavc58.134.100 pcm_s16le
size=   14592kB time=00:07:46.93 bitrate= 256.0kbits/s speed= 336x    
video:0kB audio:14592kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000522%
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-li